# Analyize Google Chrome history

Idea and initial code taken from the [Analyzing Browser History Using Python and Pandas](https://applecrazy.github.io/blog/posts/analyzing-browser-hist-using-python/) blogpost by __AppleCrazy__.

In [ ]:
%matplotlib inline

In [ ]:
import os
import pandas as pd
import numpy as np
import sqlite3
import shutil
import tempfile
import collections
from pathlib import Path
from IPython.display import display
from urllib.parse import urlparse
import matplotlib.pyplot as plt
import seaborn as sns
sns.set('notebook', style = 'white')

In [ ]:
DB_PATH = os.path.join(Path.home(), 'Library/Application Support/Google/Chrome/Default/History')

## Retrieve the history from the sqlite database

In [ ]:
def get_chrome_history_from_sqlite(db_path, filter_out_invalid_dates = True):
    TMP_DATABASE_PATH=tempfile.mktemp()
    shutil.copy(db_path, TMP_DATABASE_PATH)
    conn = sqlite3.connect(TMP_DATABASE_PATH)
    c = conn.cursor()
    
    EXPORT_CMD = "select datetime(last_visit_time/1000000-11644473600,'unixepoch'), url from  urls order by last_visit_time desc"
    
    c.execute(EXPORT_CMD)
    data = c.fetchall()
    df = pd.DataFrame(data, columns = ['datetime', 'url'])
    if filter_out_invalid_dates:
        df = df[df.datetime != '1601-01-01 00:00:00']
        
    df['domain'] = df.url.apply(lambda x: urlparse(x).netloc)
    df['datetime'] = pd.to_datetime(df.datetime)
    return df

In [ ]:
assert os.path.exists(DB_PATH), 'Chrome sqlite database "{}" does not exist!'.format(DB_PATH)
df = get_chrome_history_from_sqlite(DB_PATH)

## Plot the most frequent visited domains

This code is taken from AppleCrazy's blogpost.

In [ ]:
def get_domain_visit_counts(data):
    # Aggregate domain entries
    site_frequencies = data.domain.value_counts().to_frame()
    # Make the domain a column
    site_frequencies.reset_index(level=0, inplace=True)
    # Rename columns to appropriate names
    site_frequencies.columns = ['domain', 'count']
    return site_frequencies

def plot_domain_visit_counts_as_piechart(site_frequencies, with_labels = True, topN = 20, figsize = (14, 14)):
    fig, ax = plt.subplots(figsize=figsize)
    ax.set_title('Top {} Sites Visited\n({} visits in total)'.format(topN, site_frequencies['count'].sum()))
    pie_data = site_frequencies['count'].head(topN).tolist()

    if with_labels:
        pie_labels = site_frequencies.apply(lambda x: '{} ({})'.format(x.domain, x['count']), axis = 1).head(topN)
    else:
        pie_labels = None
    
    ax.pie(pie_data, autopct='%1.1f%%', labels=pie_labels)
    return fig, ax

site_frequencies = get_domain_visit_counts(df)
fig, ax = plot_domain_visit_counts_as_piechart(site_frequencies)

## Plot the frequency of page visits per hour of the day

In [ ]:
fig, ax = plt.subplots(figsize = (20, 6))
df.datetime.dt.hour.value_counts().sort_index(ascending = True).plot(kind = 'bar', ax = ax)
ax.set_title('Visits per hour')
ax.set_xlabel('hour of the day');

## Create "clock" plot of visits per time of day

In [ ]:
def time_to_angle(t, factor = np.pi / 12):
    return (t * factor) % (2 * np.pi)

def time_to_x_y(t, stretch_x = 1, stretch_y = 1):
    angle_in_rad = time_to_angle(t)
    x, y = np.sin(angle_in_rad), np.cos(angle_in_rad)
    return x, y

def datetime_to_angle(x):
    return time_to_angle(x.datetime.hour + x.datetime.minute / 60)

df['angle_in_rad'] = df.apply(datetime_to_angle, axis = 1)
df['x'] = np.sin(df.angle_in_rad)
df['y'] = np.cos(df.angle_in_rad)

In [ ]:
variance = 0.3
def add_variance(x, variance = 0.3):
    return x + np.random.uniform(low = -variance, high = variance, size=len(df))

df['x_ran'] = add_variance(df.x)
df['y_ran'] = add_variance(df.y)

colors = plt.get_cmap('Paired').colors
num_colors = len(colors)

cmap_domain_2_idx = {domain: colors[idx % num_colors] for idx, domain in enumerate(set(df.domain.values))}
df['domain_color'] = df.domain.apply(lambda domain: cmap_domain_2_idx[domain])

fig, ax = plt.subplots(figsize = (10, 10))
df.plot(kind = 'scatter', x = 'x_ran', y = 'y_ran', ax = ax, s = 2, alpha = 0.6, c = df.domain_color)
ax.grid('off')
fig.tight_layout()

for pos, spine in ax.spines.items():
    spine.set_visible(False)

ax.get_xaxis().set_visible(False)
ax.get_yaxis().set_visible(False)

for hour in range(24):
    x, y = time_to_x_y(hour)
    ax.text(x = x, y = y, s = hour, fontdict={'horizontalalignment': 'center', 'weight': 'bold'}, color = 'red')

ax.set_title('Page visits per hour')

fig.tight_layout()
fig.savefig('data/visits_per_hour.png')